In [ ]:
#连接ERP的DB2数据库，获取TACAIA2（前台：ACAI25）,TACACM4(ACAC80非产成品),TACACMM(ACACMM),TACACJ5(废钢入库履历)
import ibm_db
import pandas as pd
connStr='DATABASE=BHBXA1;HOSTNAME=192.168.2.28;PORT=50000;PROTOCOL=TCPIP;UID=bhbxa1;PWD=bhbxa1'
con=ibm_db.connect(connStr,'','')


In [ ]:
import ibm_db_dbi

In [ ]:
conn=ibm_db_dbi.Connection(con)

In [ ]:
tacaia2=pd.read_sql('select * from tacaia2',conn,parse_dates={'APP_TRNC_TIME':'%Y%m%d%H%M%S'})

In [ ]:
from pandas.api.types import CategoricalDtype
tacaia2['ACCOUNT_TITLE_ITEM']=tacaia2['ACCOUNT_TITLE_ITEM'].astype(CategoricalDtype(['01','33','03','31','50','51']))#这里对账务代码是01-33-03-31-50-51的顺序，因为现有产出这个材料，再有后续的专用充当和投入或销售，这里暂不考虑一个材料号被二次以上专用的情况，如上一个木材料转用经过工序加工后，转用产出一个材料号，当期这个材料又直接被专用挂了另外的订单号的情况
tacaia2['ACCOUNT_TITLE_ITEM'].cat.categories

In [ ]:
tacaia2.set_index(['ACCOUNT_TITLE_ITEM','MAT_NO','UNIT_CODE','SALE_ORDER_SUB_NO'],inplace=True) #降账务代码提到顺序第1位，以方便下面在同一投入账务代码下，按有投入的材料号迭代。

In [ ]:
tacaia2.index.names

In [ ]:
#tacaia2.sort_index(level=['ACCOUNT_TITLE_ITEM','MAT_NO','UNIT_CODE','SALE_ORDER_SUB_NO'],inplace=True) #跟着前面重设索引，把账务代码提到第1位，对应改变索引排序

In [ ]:
tacaia2.sort_index(level='MAT_NO',inplace=True) #如果按多层索引综合排序，不行，则单按账务代码排序

In [ ]:
from numpy import nan as NA
tacaia2=tacaia2.dropna() #剔除第一行的空行

In [ ]:
#tacaia2.loc['31','KEY_SEQ']

In [ ]:
tacaia2.index.get_level_values('MAT_NO')[0]

In [ ]:
#tacaia2.index.get_level_values('MAT_NO')[0]==' '  #因为排序后第一行[0]为空行，所以迭代时从第二行[1]开始（上面已删除空行，不用从第二行开始了）
for mat in tacaia2.loc['31'].index.get_level_values('MAT_NO'):
    print(tacaia2.loc[('31',mat),'KEY_SEQ'])

In [ ]:
tacaia2.to_excel('tacaia2-202005.xlsx')

In [ ]:
#tacaia2.to_excel('tacaia2-202005.xlsx',sheet_name='tacaia2')
#tacaia2.loc[('B2003125AFA','LF03','33','MD2005061901'),'PRODUCT_CODE'][0][:4] #Pandas多层索引的引用
#[x[0] for x in tacaia2.index] #多层索引中一层所有元素的列表

In [ ]:
acc_period=tacaia2['ACCOUNT_PERIOD'][1]
print(acc_period)

In [ ]:
tacacm4=pd.read_sql('select * from tacacm4 where YEAR=? and MON=?',conn,params=[acc_period[:4],acc_period[4:]])

In [ ]:
tacacmm=pd.read_sql('select * from tacacmm where YEAR=? and MON=?',conn,params=[acc_period[:4],acc_period[4:]])

In [ ]:
tacacj5=pd.read_sql_query('select * from tacacj5 where YEAR_MON=?',conn,params=[acc_period])

In [ ]:
print(tacacj5.head())
print(tacacm4.head())
print(tacacmm.head())

In [ ]:
#将ACAI25表按1、材料号升序；2、账务代码：按33 03 31 01 50 51 的顺序排列；3、销售合同号按升序排列
#tacaia2[tacaia2['MAT_WT'].rank(method='dense',ascending=False)==3]

In [ ]:
#对账务代码自定义排序：先设排序的列表；将账务代码列的数据类型改成‘category’（astype)；用set_category，设置账务代码的顺序；用sort_values使排序生效，同时，也将材料号和销售合同升序排列
acc_ttl_list=['33','03','31','01','50','51']
tacaia2['ACCOUNT_TITLE_ITEM']=tacaia2['ACCOUNT_TITLE_ITEM'].astype('category')
tacaia2['ACCOUNT_TITLE_ITEM'].cat.set_category(acc_ttl_list,inplace=True)
tacaia2.sortvalues(['MAT_NO','ACCOUNT_TITLE_ITEM','SALE_ORDER_SUB_NO'],ascending=True,inplace=True)
tacaia2